<a href="https://colab.research.google.com/github/vishnu8096/Language-Translation-Using-Transformers/blob/main/languagetranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install torchtext==0.6.0
!python -m pip install einops

     |████████████████████████████████| 1.6 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 1.2 MB 10.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [ ]:
from einops import rearrange

In [ ]:
!python -m pip install spacy

In [ ]:
!python -m spacy download en
!python -m spacy download fr
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.7 MB 5.5 MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-py3-none-any.whl size=14727026 sha256=4777120b98139b3c2d256c6c1f6493b07990d28ba34dd03d50e676e009aa4c60
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dmu67so/wheels/c9/a6/ea/0778337c34660027ee67ef3a91fb9d3600b76777a912ea1c24
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/fr
You can now load the model via spacy.loa

In [ ]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator

In [ ]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 1.33MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 231kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 223kB/s]


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
# Printing a list of tokens mapping integer to strings
print(SRC.vocab.itos)
# Printing a dict mapping tokens to indices
print(SRC.vocab.stoi)
# Printing the index of an actual word
print(SRC.vocab.stoi['ein'])

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',', 'und', 'mit', 'auf', 'mann', 'einer', 'der', 'frau', 'die', 'zwei', 'einen', 'im', 'an', 'von', 'sich', 'dem', 'mädchen', 'junge', 'vor', 'zu', 'steht', 'männer', 'sitzt', 'hund', 'den', 'straße', 'während', 'gruppe', 'hält', 'spielt', 'das', 'hemd', 'personen', 'über', 'drei', 'eines', 'frauen', 'blauen', 'neben', 'ist', 'kind', 'roten', 'weißen', 'stehen', 'sitzen', 'menschen', 'am', 'aus', 'spielen', 'durch', 'bei', 'geht', 'trägt', 'fährt', 'wasser', 'um', 'kinder', 'kleines', 'person', 'macht', 'springt', 'kleiner', 'schwarzen', 'entlang', 'leute', 'gehen', 'etwas', 'mehrere', 'seinem', 'großen', 'oberteil', 'jungen', 'hand', 'grünen', 'läuft', 'sind', 'für', 'hintergrund', 'fahrrad', 'freien', 'jacke', 'luft', 'strand', 'ball', 'hat', 'anderen', 'schaut', 'junger', 'kleidung', 'hinter', 'sie', 'nach', 'andere', 'gelben', 'kleine', 'gebäude', 'vier', 'hut', 'tisch', 'wird', 'beim', 'nähe', 'essen', 'klein

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import math

INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(INPUT_DIM, ENC_EMB_DIM)
        self.tgt_embedding = nn.Embedding(INPUT_DIM, ENC_EMB_DIM)
        self.transformer = nn.Transformer(nhead=8, num_encoder_layers=2, d_model=ENC_EMB_DIM)
        self.linear = nn.Linear(ENC_EMB_DIM, OUTPUT_DIM)
        pos_dropout = 0.1
        max_seq_length = 128
        self.pos_enc = PositionalEncoding(ENC_EMB_DIM, pos_dropout, max_seq_length)
    
    def forward(self, src, tgt, teacher_forcing_ratio=0.5, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None, tgt_mask=None):
        src_emb = self.pos_enc(self.src_embedding(src) * math.sqrt(ENC_EMB_DIM))
        tgt_emb = self.pos_enc(self.tgt_embedding(tgt) * math.sqrt(ENC_EMB_DIM))
        out = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        out = self.linear(out)
        return out

model = TransformerModel().to(device)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_normal_(p)


optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98))


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,823,077 trainable parameters


In [ ]:
# Checking index of special tokens
PAD_IDX = TRG.vocab.stoi['<pad>']
SOS_IDX = TRG.vocab.stoi['<sos>']
EOS_IDX = TRG.vocab.stoi['<eos>']
UNK_IDX = TRG.vocab.stoi['<unk>']
print('pad index:', PAD_IDX)
print('sos index:', SOS_IDX)
print('eos index:', EOS_IDX)
print('unk index:', UNK_IDX)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

pad index: 1
sos index: 2
eos index: 3
unk index: 0


In [ ]:
def gen_nopeek_mask(length):
    mask = rearrange(torch.triu(torch.ones(length, length)) == 1, 'h w -> w h')
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    return mask

In [ ]:
def example_translate(model, example_sentence_src):
    # Translate example sentence
    example_tensor_src = string_to_indices(SRC, example_sentence_src).view(-1, 1)
    example_sentence_tgt = '<sos>'
    example_tensor_tgt = string_to_indices(TRG, example_sentence_tgt).view(-1, 1)
    src = example_tensor_src.to(device)
    tgt = example_tensor_tgt.to(device)

    print('----------Translating--------------')
    for i in range(128):
        print('Src:', src)
        print('Tgt:', tgt)
        src_key_padding_mask = src == PAD_IDX
        tgt_key_padding_mask = tgt == PAD_IDX
        memory_key_padding_mask = src_key_padding_mask.clone()
        src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
        tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
        memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
        tgt_mask = gen_nopeek_mask(tgt.shape[0]).to('cuda')
        print('Tgt mask:', tgt_mask)

        output = model(src, tgt, 0, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask) #turn off teacher forcing

        print('Output:', output)
        # TODO: Check that the argmax line is correct
        output_index = torch.argmax(output, dim=2)[-1].item()
        output_word = TRG.vocab.itos[output_index]
        example_sentence_tgt = example_sentence_tgt + ' ' + output_word
        print('Translated sentence so far:', example_sentence_tgt)
        example_tensor_tgt = string_to_indices(TRG, example_sentence_tgt).view(-1, 1)
        tgt = example_tensor_tgt.to(device)
        if output_word == '<eos>':
            break
    print('-----------Finished Translating--------------')

In [ ]:
import math
import time

def indices_to_string(LANGUAGE, batch):
    words_list = []
    for sentence in batch.transpose(1, 0):
        sentence_list = sentence.tolist()
        words = []
        for index in sentence_list:
            word = LANGUAGE.vocab.itos[index]
            words.append(word)
        words_list.append(words)
    return words_list

def string_to_indices(LANGUAGE, sentence):
    words = sentence.split()
    indices = []
    for word in words:
        if word in LANGUAGE.vocab.stoi:
            index = LANGUAGE.vocab.stoi[word]
            indices.append(index)
        else:
            index = LANGUAGE.vocab.stoi['<unk>']
            indices.append(index)
    result = torch.tensor(indices)
    return result

def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        tgt = batch.trg

        optimizer.zero_grad()

        # Original src shape: (sentence length=24, batch_size=128)
        # Transformer expects: (sentence length=24, batch_size=128, embedding_size=128)
        src_key_padding_mask = src == PAD_IDX
        tgt_key_padding_mask = tgt == PAD_IDX
        memory_key_padding_mask = src_key_padding_mask.clone()
        src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
        tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
        memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
        tgt_sentence_len = tgt.shape[0] - torch.sum(tgt_key_padding_mask, axis=1)
        tgt_inp, tgt_out = tgt[:-1, :], tgt[1:, :]
        tgt_key_padding_mask = tgt_key_padding_mask[:, :-1]
        tgt_mask = gen_nopeek_mask(tgt_inp.shape[0]).to('cuda')
        output = model(src, tgt_inp, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask)
        from_one_hot = torch.argmax(output, dim=2)
        # output shape: (sentence length=24, batch_size=128, vocab=5893)
        # Original tgt shape: (sentence length=24, batch_size=128)

        output = output.view(-1, output.shape[-1])
        tgt_out = tgt_out.view(-1)

        loss = criterion(output, tgt_out)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)



def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    print('Evaluating')
    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            tgt = batch.trg

            src_key_padding_mask = src == PAD_IDX
            tgt_key_padding_mask = tgt == PAD_IDX
            memory_key_padding_mask = src_key_padding_mask.clone()
            src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
            tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
            memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
            tgt_mask = gen_nopeek_mask(tgt.shape[0]).to('cuda')
            output = model(src, tgt, 0, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask) #turn off teacher forcing
            from_one_hot = torch.argmax(output, dim=2)
            #print(src.shape, output.shape, from_one_hot.shape, tgt.shape)

            output = output[1:].view(-1, output.shape[-1])
            #print('Src:', src.transpose(1, 0))
            #print('Predicted:', from_one_hot.transpose(1, 0))
            #print('Target:', tgt.transpose(1, 0))
            src_words = indices_to_string(SRC, src)
            predicted_words = indices_to_string(TRG, from_one_hot)
            tgt_words = indices_to_string(TRG, tgt)
            #for i, src in enumerate(src_words):
            #    print('----------------------------------')
            #    print(' '.join(src))
            #    print(' '.join(predicted_words[i]))
            #    print(' '.join(tgt_words[i]))

            tgt = tgt[1:].view(-1)
            loss = criterion(output, tgt)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Evaluating
Epoch: 01 | Time: 0m 34s
	Train Loss: 5.852 | Train PPL: 347.760
	 Val. Loss: 5.870 |  Val. PPL: 354.199
Evaluating
Epoch: 02 | Time: 0m 34s
	Train Loss: 4.124 | Train PPL:  61.796
	 Val. Loss: 6.859 |  Val. PPL: 952.717
Evaluating
Epoch: 03 | Time: 0m 34s
	Train Loss: 3.616 | Train PPL:  37.198
	 Val. Loss: 7.139 |  Val. PPL: 1260.317
Evaluating
Epoch: 04 | Time: 0m 34s
	Train Loss: 3.293 | Train PPL:  26.925
	 Val. Loss: 7.374 |  Val. PPL: 1593.860
Evaluating
Epoch: 05 | Time: 0m 34s
	Train Loss: 3.034 | Train PPL:  20.772
	 Val. Loss: 7.816 |  Val. PPL: 2480.775
Evaluating
Epoch: 06 | Time: 0m 34s
	Train Loss: 2.809 | Train PPL:  16.587
	 Val. Loss: 8.114 |  Val. PPL: 3339.595
Evaluating
Epoch: 07 | Time: 0m 34s
	Train Loss: 2.624 | Train PPL:  13.788
	 Val. Loss: 8.420 |  Val. PPL: 4537.211
Evaluating
Epoch: 08 | Time: 0m 34s
	Train Loss: 2.474 | Train PPL:  11.869
	 Val. Loss: 8.417 |  Val. PPL: 4521.242
Evaluating
Epoch: 09 | Time: 0m 34s
	Train Loss: 2.348 | Train PPL

In [ ]:
test_loss = evaluate(model, test_iterator, criterion)

Evaluating


In [ ]:
example_sentence_src = '<sos> der himmel ist blau <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'
#example_sentence_src = '<sos> viele menschen haben sich versammelt , um etwas zu sehen , das nicht auf dem foto ist . <eos> <pad> <pad> <pad> <pad>'
example_translate(model, example_sentence_src)

----------Translating--------------
Src: tensor([[  2],
        [ 15],
        [368],
        [ 48],
        [234],
        [  3],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1]], device='cuda:0')
Tgt: tensor([[2]], device='cuda:0')
Tgt mask: tensor([[0.]], device='cuda:0')
Output: tensor([[[  4.5196, -12.4259, -12.4903,  ...,  -3.1531,  -5.2270,  -4.9221]]],
       device='cuda:0', grad_fn=<AddBackward0>)
Translated sentence so far: <sos> the
Src: tensor([[  2],
        [ 15],
        [368],
        [ 48],
        [234],
        [  3],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1],
        [  1]], device='cuda:0')
Tgt: tensor([[2],
        [7]], device='cuda:0')
Tgt mask: tensor([[0., -inf],
        [0., 0.]], device='cuda:0')
Output: tensor([[[  4.5196, -12.4259, -12.4903,  ...,  -3.1531,  -5.2270